In [ ]:
!pip install datasets transformers trl evaluate huggingface_hub pynvml psutil rouge_score trackio

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 462.8/462.8 kB 28.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 875.0/875.0 kB 45.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 115.4 MB/s eta 0:00:00
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=bfef6e8fd5aca7be5f686f02231108adb3bdf3416b840555a841d0ab9433dda8
  Stored in directory: /root/.cache/pip/wheels/85/9d/af/01feefbe7d55ef5468796f0c68225b6788e85d9d0a281e7a70
Successfully built rouge_score
  Attempting uninstall: plotly
    Found existing installation: plotly 5.24.1
    Uninstalling plotly-5.24.1:
      Successfully uninstalled plotly-5.24.1


In [ ]:


from datasets import load_dataset
from peft import LoraConfig
from trl import SFTTrainer, SFTConfig
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
import evaluate
from transformers import TrainerCallback
import time
from huggingface_hub import login
from pynvml import nvmlInit, nvmlDeviceGetHandleByIndex, nvmlDeviceGetMemoryInfo
import pynvml
import psutil

print("login")

login(token="YourToken")

print("data is loading")

# Load full FINDSum dataset
ds_train = load_dataset('csv', data_files='./drive/MyDrive/text_only_train_data.csv')
ds_val = load_dataset('csv', data_files={'validation':'./drive/MyDrive/text_only_val_data.csv'})

train = ds_train["train"]

val = ds_val["validation"]

def add_prefix(example):
    prefix = "Summarize the following document: "
    example["document"] = prefix + example["document"]
    return example

train = train.map(add_prefix)
val = val.map(add_prefix)

print("model loading")

model_id_mistral = "mistralai/Mistral-7B-Instruct-v0.3"
model_ids = [{"id": model_id_mistral, "name": "mistral_4"}]

model_tokenizer_list = []

for model_id in model_ids:
    tokenizer = AutoTokenizer.from_pretrained(model_id["id"])
    model = AutoModelForCausalLM.from_pretrained(model_id["id"], dtype=torch.bfloat16, device_map="auto")
    model_tokenizer_list.append({
        "model": model,
        "tokenizer": tokenizer,
        "name": model_id["name"]
    })

print("Loading ROUGE metric...")
rouge = evaluate.load("rouge")

def compute_metrics(eval_preds):
    predictions, labels = eval_preds
    # decode token IDs into strings if needed (tokenizer needed here)
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Run rouge evaluation on decoded text
    result = rouge.compute(predictions=decoded_preds, references=decoded_labels)
    print("ROUGE Log", result)
    return {
        "rouge1": result["rouge1"],
        "rouge2": result["rouge2"],
        "rougeL": result["rougeL"]
    }
class ResourceLoggingCallback(TrainerCallback):
    def __init__(self, log_interval=60):
        super().__init__()
        self.log_interval = log_interval  # seconds
        self.last_log_time = time.time()
        nvmlInit()
        self.handle = nvmlDeviceGetHandleByIndex(0)  # GPU 0

    def on_step_end(self, args, state, control, **kwargs):
        current_time = time.time()
        if current_time - self.last_log_time >= self.log_interval:
            # Log GPU memory
            mem_info = nvmlDeviceGetMemoryInfo(self.handle)
            gpu_used = mem_info.used / (1024 ** 3)
            gpu_total = mem_info.total / (1024 ** 3)

            # Log CPU/RAM usage
            cpu_percent = psutil.cpu_percent()
            ram_info = psutil.virtual_memory()

            print(f"[Resource Log | Step {state.global_step}] "
                  f"GPU Memory Used: {gpu_used:.2f} GB / {gpu_total:.2f} GB, "
                  f"CPU Usage: {cpu_percent}%, "
                  f"RAM Usage: {ram_info.used / (1024 ** 3):.2f} GB / {ram_info.total / (1024 ** 3):.2f} GB")

            self.last_log_time = current_time

    def on_train_end(self, args, state, control, **kwargs):
        total_flos = getattr(state, 'total_flos', None)
        if total_flos is not None:
            print(f"Total FLOPs during training: {total_flos}")
        else:
            print("Total FLOPs information not available.")

print(model_id)
print("training")
peft_config = LoraConfig(
    r=64,
    lora_alpha=64,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.1,
    bias="none",
    task_type="CAUSAL_LM")

training_args = SFTConfig(
    dataset_text_field="document",
    learning_rate=2e-4,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=64,
    num_train_epochs=2,
    report_to=["trackio"],
    save_strategy="epoch" ,  # Save checkpoint at end of each epoch
    gradient_checkpointing=True
)
for models in model_tokenizer_list:
    print(models["name"] +"started training")
    training_args.output_dir = f"./drive/MyDrive/Results_Lora/Mistral/sft-lora-{models['name']}"
    trainer = SFTTrainer(
        model=model,
        train_dataset=train,
        eval_dataset=val,
        peft_config=peft_config,
        args=training_args,
        compute_metrics=compute_metrics,
        callbacks=[ResourceLoggingCallback(log_interval=120)],
        processing_class=tokenizer  # Pass tokenizer here, not as "tokenizer"
    )

    trainer.train()
    trainer.save_model("./drive/MyDrive/Results_Lora/Mistral/my_trained__mistral_model")
    trainer.save_state()
    print(models["name"] +"finished training")

login
data is loading


Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/16820 [00:00<?, ? examples/s]

Map:   0%|          | 0/2102 [00:00<?, ? examples/s]

...

The model is already on multiple devices. Skipping the move to device specified in `args`.


* Trackio project initialized: huggingface
* Trackio metrics will be synced to Hugging Face Dataset: Dali-bot/trackio-dataset
* Found existing space: https://huggingface.co/spaces/Dali-bot/trackio
* View dashboard by going to: https://Dali-bot-trackio.hf.space/


* Created new run: Dali-bot-1762784594
[Resource Log | Step 1] GPU Memory Used: 15.72 GB / 40.00 GB, CPU Usage: 14.6%, RAM Usage: 5.39 GB / 83.47 GB


Step,Training Loss
10,1.579200
20,1.490500
30,1.474000
40,1.466400
50,1.467200
60,1.459000
70,1.467000
80,1.437500
90,1.437100
100,1.444100


[Resource Log | Step 8] GPU Memory Used: 15.77 GB / 40.00 GB, CPU Usage: 10.5%, RAM Usage: 5.37 GB / 83.47 GB
[Resource Log | Step 15] GPU Memory Used: 15.77 GB / 40.00 GB, CPU Usage: 10.4%, RAM Usage: 5.38 GB / 83.47 GB
[Resource Log | Step 22] GPU Memory Used: 15.77 GB / 40.00 GB, CPU Usage: 10.5%, RAM Usage: 5.36 GB / 83.47 GB
[Resource Log | Step 29] GPU Memory Used: 15.77 GB / 40.00 GB, CPU Usage: 10.4%, RAM Usage: 5.36 GB / 83.47 GB
[Resource Log | Step 36] GPU Memory Used: 15.77 GB / 40.00 GB, CPU Usage: 10.5%, RAM Usage: 5.34 GB / 83.47 GB
[Resource Log | Step 43] GPU Memory Used: 15.77 GB / 40.00 GB, CPU Usage: 10.4%, RAM Usage: 5.33 GB / 83.47 GB
[Resource Log | Step 50] GPU Memory Used: 15.77 GB / 40.00 GB, CPU Usage: 10.4%, RAM Usage: 5.35 GB / 83.47 GB
[Resource Log | Step 57] GPU Memory Used: 15.77 GB / 40.00 GB, CPU Usage: 10.6%, RAM Usage: 5.33 GB / 83.47 GB
[Resource Log | Step 64] GPU Memory Used: 15.77 GB / 40.00 GB, CPU Usage: 10.4%, RAM Usage: 5.34 GB / 83.47 GB
[R

In [ ]:


from datasets import load_dataset
from peft import LoraConfig
from trl import SFTTrainer, SFTConfig
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
import evaluate
from transformers import TrainerCallback
import time
from huggingface_hub import login
from pynvml import nvmlInit, nvmlDeviceGetHandleByIndex, nvmlDeviceGetMemoryInfo
import pynvml
import psutil

print("login")

login(token="YourToken")

print("data is loading")

# Load full FINDSum dataset
ds_train = load_dataset('csv', data_files='./drive/MyDrive/text_only_train_data.csv')
ds_val = load_dataset('csv', data_files={'validation':'./drive/MyDrive/text_only_val_data.csv'})

train = ds_train["train"]

val = ds_val["validation"]

def add_prefix(example):
    prefix = "Summarize the following document: "
    example["document"] = prefix + example["document"]
    return example

train = train.map(add_prefix)
val = val.map(add_prefix)

print("model loading")

model_id_qwen2_5 = "Qwen/Qwen2.5-7B-Instruct"
model_ids = [{"id": model_id_qwen2_5, "name": "qwen_4"}]

model_tokenizer_list = []

for model_id in model_ids:
    tokenizer = AutoTokenizer.from_pretrained(model_id["id"])
    model = AutoModelForCausalLM.from_pretrained(model_id["id"], dtype=torch.bfloat16, device_map="auto")
    model_tokenizer_list.append({
        "model": model,
        "tokenizer": tokenizer,
        "name": model_id["name"]
    })

print("Loading ROUGE metric...")
rouge = evaluate.load("rouge")

def compute_metrics(eval_preds):
    predictions, labels = eval_preds
    # decode token IDs into strings if needed (tokenizer needed here)
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Run rouge evaluation on decoded text
    result = rouge.compute(predictions=decoded_preds, references=decoded_labels)
    print("ROUGE Log", result)
    return {
        "rouge1": result["rouge1"],
        "rouge2": result["rouge2"],
        "rougeL": result["rougeL"]
    }
class ResourceLoggingCallback(TrainerCallback):
    def __init__(self, log_interval=60):
        super().__init__()
        self.log_interval = log_interval  # seconds
        self.last_log_time = time.time()
        nvmlInit()
        self.handle = nvmlDeviceGetHandleByIndex(0)  # GPU 0

    def on_step_end(self, args, state, control, **kwargs):
        current_time = time.time()
        if current_time - self.last_log_time >= self.log_interval:
            # Log GPU memory
            mem_info = nvmlDeviceGetMemoryInfo(self.handle)
            gpu_used = mem_info.used / (1024 ** 3)
            gpu_total = mem_info.total / (1024 ** 3)

            # Log CPU/RAM usage
            cpu_percent = psutil.cpu_percent()
            ram_info = psutil.virtual_memory()

            print(f"[Resource Log | Step {state.global_step}] "
                  f"GPU Memory Used: {gpu_used:.2f} GB / {gpu_total:.2f} GB, "
                  f"CPU Usage: {cpu_percent}%, "
                  f"RAM Usage: {ram_info.used / (1024 ** 3):.2f} GB / {ram_info.total / (1024 ** 3):.2f} GB")

            self.last_log_time = current_time

    def on_train_end(self, args, state, control, **kwargs):
        total_flos = getattr(state, 'total_flos', None)
        if total_flos is not None:
            print(f"Total FLOPs during training: {total_flos}")
        else:
            print("Total FLOPs information not available.")

print(model_id)
print("training")
peft_config = LoraConfig(
    r=64,
    lora_alpha=16,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.1,
    bias="none",
    task_type="CAUSAL_LM")

training_args = SFTConfig(
    dataset_text_field="document",
    learning_rate=2e-4,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=64,
    num_train_epochs=2,
    report_to=["trackio"],
    save_strategy="epoch" ,  # Save checkpoint at end of each epoch
    gradient_checkpointing=True
)
for models in model_tokenizer_list:
    print(models["name"] +"started training")
    training_args.output_dir = f"./drive/MyDrive/Results_Lora/Qwen/sft-lora-{models['name']}"
    trainer = SFTTrainer(
        model=model,
        train_dataset=train,
        eval_dataset=val,
        peft_config=peft_config,
        args=training_args,
        compute_metrics=compute_metrics,
        callbacks=[ResourceLoggingCallback(log_interval=120)],
        processing_class=tokenizer  # Pass tokenizer here, not as "tokenizer"
    )

    trainer.train()
    trainer.save_model("./drive/MyDrive/Results_Lora/Qwen/my_trained_qwen_model")
    trainer.save_state()
    print(models["name"] +"finished training")

login
data is loading


Map:   0%|          | 0/16820 [00:00<?, ? examples/s]

Map:   0%|          | 0/2102 [00:00<?, ? examples/s]

model loading


tokenizer_config.json: 0.00B [00:00, ?B/s]

...

Truncating eval dataset:   0%|          | 0/2102 [00:00<?, ? examples/s]

The model is already on multiple devices. Skipping the move to device specified in `args`.
The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'bos_token_id': None, 'pad_token_id': 151643}.


* Created new run: Dali-bot-1762796197
[Resource Log | Step 1] GPU Memory Used: 17.92 GB / 40.00 GB, CPU Usage: 5.1%, RAM Usage: 7.99 GB / 83.47 GB


Step,Training Loss
10,1.742800
20,1.666200
30,1.627100
40,1.617200
50,1.607800
60,1.601900
70,1.600800
80,1.583000
90,1.582800
100,1.579700


[Resource Log | Step 8] GPU Memory Used: 17.96 GB / 40.00 GB, CPU Usage: 10.6%, RAM Usage: 7.98 GB / 83.47 GB
[Resource Log | Step 15] GPU Memory Used: 17.96 GB / 40.00 GB, CPU Usage: 10.4%, RAM Usage: 7.97 GB / 83.47 GB
[Resource Log | Step 22] GPU Memory Used: 17.96 GB / 40.00 GB, CPU Usage: 10.6%, RAM Usage: 7.97 GB / 83.47 GB
[Resource Log | Step 29] GPU Memory Used: 17.96 GB / 40.00 GB, CPU Usage: 10.6%, RAM Usage: 7.97 GB / 83.47 GB
[Resource Log | Step 36] GPU Memory Used: 17.96 GB / 40.00 GB, CPU Usage: 10.4%, RAM Usage: 7.97 GB / 83.47 GB
[Resource Log | Step 43] GPU Memory Used: 17.96 GB / 40.00 GB, CPU Usage: 10.6%, RAM Usage: 7.97 GB / 83.47 GB
[Resource Log | Step 50] GPU Memory Used: 17.96 GB / 40.00 GB, CPU Usage: 10.6%, RAM Usage: 7.97 GB / 83.47 GB
[Resource Log | Step 57] GPU Memory Used: 17.96 GB / 40.00 GB, CPU Usage: 10.4%, RAM Usage: 7.94 GB / 83.47 GB
[Resource Log | Step 64] GPU Memory Used: 17.96 GB / 40.00 GB, CPU Usage: 10.6%, RAM Usage: 7.95 GB / 83.47 GB
[R

In [ ]:


from datasets import load_dataset
from peft import LoraConfig
from trl import SFTTrainer, SFTConfig
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
import evaluate
from transformers import TrainerCallback
import time
from huggingface_hub import login
from pynvml import nvmlInit, nvmlDeviceGetHandleByIndex, nvmlDeviceGetMemoryInfo
import pynvml
import psutil

print("login")

login(token="YourToken")

print("data is loading")

# Load full FINDSum dataset
ds_train = load_dataset('csv', data_files='./drive/MyDrive/text_only_train_data.csv')
ds_val = load_dataset('csv', data_files={'validation':'./drive/MyDrive/text_only_val_data.csv'})

train = ds_train["train"]

val = ds_val["validation"]

def add_prefix(example):
    prefix = "Summarize the following document: "
    example["document"] = prefix + example["document"]
    return example

train = train.map(add_prefix)
val = val.map(add_prefix)

print("model loading")

model_id_cohereLabs = "CohereLabs/c4ai-command-r7b-12-2024"
model_ids = [{"id": model_id_cohereLabs, "name": "coherelabs_4"}]

model_tokenizer_list = []

for model_id in model_ids:
    tokenizer = AutoTokenizer.from_pretrained(model_id["id"])
    model = AutoModelForCausalLM.from_pretrained(model_id["id"], dtype=torch.bfloat16, device_map="auto")
    model_tokenizer_list.append({
        "model": model,
        "tokenizer": tokenizer,
        "name": model_id["name"]
    })

print("Loading ROUGE metric...")
rouge = evaluate.load("rouge")

def compute_metrics(eval_preds):
    predictions, labels = eval_preds
    # decode token IDs into strings if needed (tokenizer needed here)
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Run rouge evaluation on decoded text
    result = rouge.compute(predictions=decoded_preds, references=decoded_labels)
    print("ROUGE Log", result)
    return {
        "rouge1": result["rouge1"],
        "rouge2": result["rouge2"],
        "rougeL": result["rougeL"]
    }
class ResourceLoggingCallback(TrainerCallback):
    def __init__(self, log_interval=60):
        super().__init__()
        self.log_interval = log_interval  # seconds
        self.last_log_time = time.time()
        nvmlInit()
        self.handle = nvmlDeviceGetHandleByIndex(0)  # GPU 0

    def on_step_end(self, args, state, control, **kwargs):
        current_time = time.time()
        if current_time - self.last_log_time >= self.log_interval:
            # Log GPU memory
            mem_info = nvmlDeviceGetMemoryInfo(self.handle)
            gpu_used = mem_info.used / (1024 ** 3)
            gpu_total = mem_info.total / (1024 ** 3)

            # Log CPU/RAM usage
            cpu_percent = psutil.cpu_percent()
            ram_info = psutil.virtual_memory()

            print(f"[Resource Log | Step {state.global_step}] "
                  f"GPU Memory Used: {gpu_used:.2f} GB / {gpu_total:.2f} GB, "
                  f"CPU Usage: {cpu_percent}%, "
                  f"RAM Usage: {ram_info.used / (1024 ** 3):.2f} GB / {ram_info.total / (1024 ** 3):.2f} GB")

            self.last_log_time = current_time

    def on_train_end(self, args, state, control, **kwargs):
        total_flos = getattr(state, 'total_flos', None)
        if total_flos is not None:
            print(f"Total FLOPs during training: {total_flos}")
        else:
            print("Total FLOPs information not available.")

print(model_id)
print("training")
peft_config = LoraConfig(
    r=64,
    lora_alpha=64,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.1,
    bias="none",
    task_type="CAUSAL_LM")

training_args = SFTConfig(
    dataset_text_field="document",
    learning_rate=2e-4,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=64,
    num_train_epochs=2,
    report_to=["trackio"],
    save_strategy="epoch",  # Save checkpoint at end of each epoch
    gradient_checkpointing=True
)
for models in model_tokenizer_list:
    print(models["name"] +"started training")
    training_args.output_dir = f"./drive/MyDrive/Results_Lora/Command/sft-lora-{models['name']}"
    trainer = SFTTrainer(
        model=model,
        train_dataset=train,
        eval_dataset=val,
        peft_config=peft_config,
        args=training_args,
        compute_metrics=compute_metrics,
        callbacks=[ResourceLoggingCallback(log_interval=120)],
        processing_class=tokenizer  # Pass tokenizer here, not as "tokenizer"
    )

    trainer.train()
    trainer.save_model("./drive/MyDrive/Results_Lora/Command/my_trained_command_model")
    trainer.save_state()
    print(models["name"] +"finished training")

login
data is loading


Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/16820 [00:00<?, ? examples/s]

Map:   0%|          | 0/2102 [00:00<?, ? examples/s]

...

The model is already on multiple devices. Skipping the move to device specified in `args`.


* Trackio project initialized: huggingface
* Trackio metrics will be synced to Hugging Face Dataset: Dali-bot/trackio-dataset
* Found existing space: https://huggingface.co/spaces/Dali-bot/trackio
* View dashboard by going to: https://Dali-bot-trackio.hf.space/


* Created new run: Dali-bot-1762834190
[Resource Log | Step 1] GPU Memory Used: 20.62 GB / 40.00 GB, CPU Usage: 10.8%, RAM Usage: 5.27 GB / 83.47 GB


Step,Training Loss
10,1.785900
20,1.702300
30,1.660300
40,1.651300
50,1.646900
60,1.641600
70,1.644900
80,1.623500
90,1.621900
100,1.623500


[Resource Log | Step 7] GPU Memory Used: 20.67 GB / 40.00 GB, CPU Usage: 10.5%, RAM Usage: 5.31 GB / 83.47 GB
[Resource Log | Step 13] GPU Memory Used: 20.67 GB / 40.00 GB, CPU Usage: 10.5%, RAM Usage: 5.30 GB / 83.47 GB
[Resource Log | Step 19] GPU Memory Used: 20.67 GB / 40.00 GB, CPU Usage: 10.5%, RAM Usage: 5.30 GB / 83.47 GB
[Resource Log | Step 25] GPU Memory Used: 20.67 GB / 40.00 GB, CPU Usage: 10.5%, RAM Usage: 5.30 GB / 83.47 GB
[Resource Log | Step 31] GPU Memory Used: 20.67 GB / 40.00 GB, CPU Usage: 10.6%, RAM Usage: 5.29 GB / 83.47 GB
[Resource Log | Step 37] GPU Memory Used: 20.67 GB / 40.00 GB, CPU Usage: 10.6%, RAM Usage: 5.29 GB / 83.47 GB
[Resource Log | Step 43] GPU Memory Used: 20.67 GB / 40.00 GB, CPU Usage: 10.5%, RAM Usage: 5.30 GB / 83.47 GB
[Resource Log | Step 49] GPU Memory Used: 20.67 GB / 40.00 GB, CPU Usage: 10.4%, RAM Usage: 5.29 GB / 83.47 GB
[Resource Log | Step 55] GPU Memory Used: 20.67 GB / 40.00 GB, CPU Usage: 10.5%, RAM Usage: 5.26 GB / 83.47 GB
[R